## 학습된 모델을 통해 레시피 벡터 추출

In [ ]:
# 식재료 RNN 모델 정의
class RNNNoun(nn.Module):
    def __init__(self, input_size, hidden_size, output_size):
        super(RNNNoun, self).__init__()
        self.hidden_size = hidden_size
        self.dense = nn.Linear(input_size, hidden_size)  # Dense layer
        self.rnn = nn.RNN(hidden_size, hidden_size, batch_first=True)
        self.fc = nn.Linear(hidden_size, output_size)
        self.softmax = nn.LogSoftmax(dim=1)

    def forward(self, x):
        x = self.dense(x)
        h0 = torch.zeros(1, x.size(0), self.hidden_size)
        out, _ = self.rnn(x, h0)
        out1 = out[:, -1, :]
        out2 = self.fc(out1)
        out3 = self.softmax(out2)
        return out1, out2, out3
    
# 저장된 모델 불러오기
checkpoint = torch.load('model_checkpoint6.pth')

# 모델과 옵티마이저 초기화
model_noun = RNNNoun(checkpoint['input_size'], checkpoint['hidden_size'], checkpoint['output_size'])
model_noun = model_noun.float()

optimizer_noun = torch.optim.Adam(model_noun.parameters(), lr=0.001)
criterion = nn.CrossEntropyLoss()

# 불러온 모델의 상태와 옵티마이저 상태 적용
model_noun.load_state_dict(checkpoint['model_state_dict'])
optimizer_noun.load_state_dict(checkpoint['optimizer_state_dict'])

In [ ]:
# 식재료 벡터 불러오기
import pickle
with open('ingre.pickle', 'rb') as f:
    test = pickle.load(f)
    
# 식재료 벡터 불러오기
import pickle
with open('sliced_verb_noun.pkl', 'rb') as f:
    sliced_verb_noun = pickle.load(f)

In [ ]:
# 100차원의 0으로 채워진 배열 생성
dim = 100
zero_array = np.zeros(dim)

# 단어 사전의 크기와 임베딩 차원을 정의합니다.
vocab_size = len(unique_stems)  # 단어 사전의 크기
embedding_dim = 100  # 임베딩 차원

# 임베딩 레이어 초기화
embedding = nn.Embedding(vocab_size, embedding_dim)

# 슬라이싱된 데이터 벡터로 변환
total_list = []
for sliced_recipe in tqdm(sliced_verb_noun):
    recipe_vec = []
    for token_list in sliced_recipe:
        sliced_vec = []
        for token in token_list:
            
            # 조리방법 사전에 있는 동사 토큰만 가져오기
            if token in unique_stems:
                index = unique_stems.index(token)
                word_index = torch.LongTensor([index])
                word_embed = embedding(word_index) # 임베딩 레이어에 통과시켜 식재료 차원(100차원)과 맞춰주기
                word_embed = word_embed.squeeze(0)
                sliced_vec.append(word_embed.detach().numpy())
            
            # 식재료는 미리 만들어둔 100차원으로 변경
            elif token in test:
                food_token = test[token]
                sliced_vec.append(food_token)
            
            # 조리방법 사전과 식재료 사전에 모두 없는 토큰은 0으로 이루어진 100차원 어레이 추가
            elif token == '':
                sliced_vec.append(zero_array)

        recipe_vec.append(sliced_vec)
    total_list.append(recipe_vec)

In [ ]:
# 각 레시피 벡터 구하기
recipe_tensor_list = [] # 레시피 별 슬라이스 된 리스트를 모델에 넣은 결과를 추가할 리스트
for recipe in tqdm(total_list):
    step_tensor_list = []
    for step in recipe:
        step = torch.tensor(step).unsqueeze(0)
        step = step.float()
        step = step.squeeze(-1)
        out1, out2, out3 = model_noun(step)
        step_tensor_list.append(out1)
    mean_tensor = torch.mean(torch.stack(step_tensor_list), dim=0) # 레시피 별 슬라이스 된 리스트의 결과값 평균
    recipe_tensor_list.append(mean_tensor) # 평균 낸 벡터를 레시피 벡터로 간주

In [ ]:
# 레시피별 벡터 파일에 저장 (이진 파일로 저장)
with open('recipe_tensor_list.pkl', 'wb') as file:
    pickle.dump(recipe_tensor_list, file)